# Visualize light curves for all the sources of a region and skymaps of its fields 

In [1]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

from regions import CircleSkyRegion
from astropy import units as uu
from astropy.coordinates import SkyCoord
from astropy.wcs.utils import pixel_to_skycoord

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis

from vasca.utils import mag2flux

# ------------------  Setup region and plotting paramters  ----------------------

region_name = "TDS_ELAISN1" #_ELAISN1
lc_block_srcs = 5 #Setup light curve plotting parameters
show_lc_pars = ["rg_src_id","nr_det","flux","mag","flux_cpval","flux_nxv","flux_rchiq"]
show_map_pars = ["rg_src_id","ra","dec","pos_err","pos_nxv", "pos_cpval","nr_det","flux","mag","flux_cpval", "nr_fd_srcs"]
sort_par = "rg_src_id" # "flux" #"rg_src_id" #
only_sel_srcs = True

# -------------------------------------------------------------------------------

#Load region
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
rg = Region()
rg.load_from_fits(region_fname)

tt_srcs = rg.tt_sources
if only_sel_srcs:
    tt_srcs = rg.tt_sources[rg.tt_sources["sel"]]
nr_srcs = len(tt_srcs)
tt_srcs.sort(sort_par)
tt_srcs.add_index("rg_src_id")

### Show all info for one source

In [2]:
def plot_sky_src(rg_src_id=0):
    if rg_src_id not in tt_srcs["rg_src_id"]:
        print("source ID does not exist") 
        return
    
    fig_src = plt.figure("Sky map source",figsize=(8, 7))
    plt.cla()

    tc_src = rg.get_src_from_id(rg_src_id)
    
    #Get sky region of source
    sky_center = SkyCoord(tc_src.tt_sources[0]["ra"]*uu.deg, tc_src.tt_sources[0]["dec"]*uu.deg, frame="icrs")
    sky_region_src = CircleSkyRegion(center=sky_center, radius=0.3*uu.arcmin)
    
    #Get field to plot
    fd_src = rg.get_field(rg_fd_id=tc_src.tt_fields[0]["rg_fd_id"], load_method="FITS", add_field=True)
    
    #Plot
    #plt.cla()
    ax_src, wcs_src = vvis.plot_field_sky_map(fd_src, img_idx=-1, sky_region = sky_region_src,fig=fig_src) # 
    _ = vvis.plot_sky_sources(rg.tt_sources,tt_det=rg.tt_detections, sky_region_wcs = (sky_region_src,wcs_src),only_selected=False) #
   # _ = vvis.plot_sky_sources(rg.tt_ref_sources,tt_det=None, sky_region_wcs = (sky_region_src,wcs_src),only_selected=False)
    plt.show()
    
    fig_lc_src = plt.figure("Light curve",figsize=(8, 4))
    plt.cla()
    vvis.plot_light_curve(rg,fig=fig_lc_src, rg_src_ids = rg_src_id)
    plt.show()
    
    
    display(tc_src.tt_sources[show_map_pars])
    display(tc_src.tt_detections)
    display(tc_src.tt_fields)
    
visprob_srcids = [1669,1801,2815,3776,4338,6137,]


widgets.interactive( plot_sky_src, rg_src_id=visprob_srcids) #tt_srcs["rg_src_id"]

interactive(children=(Dropdown(description='rg_src_id', options=(1669, 1801, 2815, 3776, 4338, 6137), value=16…

rg_src_id Ra dec comment
140461 150.2487081420898	2.2611373601283726  Offset from maximum 
143003 150.11600401215506	2.244982524539299   Source across field 
143491 150.0886929944955	2.244071937913435   One ourlyer 

1669	242.03737180641812	54.45854047458777  Offset from image
1801	241.98778211802238	54.13409995290732  Nearby source
2815	241.45411996509637	54.92288017258464  Looks very wrong, between two sources, flux 212
3776	245.24663109423406	55.18655340244798  Only two dets
4338	245.07276757329817	55.17424168308725  Bursty, offset, mag17.8
6137	244.73443811185908	54.46339958688894  Offset mag 20

### Draw sky maps of fields

In [3]:
def plot_sky_field(rg_fd_id):
    if rg_fd_id not in rg.tt_fields["rg_fd_id"]:
        print("source ID does not exist") 
        return
    fig_skymap = plt.figure("Sky map",figsize=(9, 8)) #, constrained_layout=True
    plt.cla()
    fd = rg.get_field(rg_fd_id=rg_fd_id, load_method="FITS", add_field=True)
    ax, wcs = vvis.plot_field_sky_map(fd, img_idx=-1, sky_region = fd.get_sky_region(),fig=fig_skymap) # 
    ax, rg_src_ids = vvis.plot_sky_sources(tt_srcs,tt_det=rg.tt_detections, sky_region_wcs = (fd.get_sky_region(),wcs),only_selected=False) # 
    src_idx = tt_srcs.loc_indices["rg_src_id", rg_src_ids]
    plt.show()
    display(tt_srcs[src_idx][show_map_pars])
widgets.interactive( plot_sky_field, rg_fd_id=(rg.tt_fields["rg_fd_id"].min(),rg.tt_fields["rg_fd_id"].max()))

interactive(children=(IntSlider(value=3, description='rg_fd_id', max=6), Output()), _dom_classes=('widget-inte…

### Draw light curves

In [4]:
nr_blocks = int(nr_srcs/lc_block_srcs)+1 
def plot_lc_range(block_nr):
    fig_lc = plt.figure("Light curves",figsize=(10, 9))
    plt.cla()
    lc_min = lc_block_srcs * block_nr
    lc_max = lc_min + lc_block_srcs if lc_min + lc_block_srcs<=nr_srcs-1 else nr_srcs-1
    vvis.plot_light_curve(rg,fig=fig_lc, rg_src_ids = tt_srcs["rg_src_id"][lc_min:lc_max], ylim = [1.2,350])
    plt.show()
    display(tt_srcs[lc_min:lc_max][show_lc_pars])
widgets.interactive( plot_lc_range, block_nr=(0,nr_blocks-1))

interactive(children=(IntSlider(value=29, description='block_nr', max=58), Output()), _dom_classes=('widget-in…